In [99]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk.corpus
from nltk.stem.snowball import SnowballStemmer
from sklearn.cross_validation import train_test_split
from ml_metrics import rmsle
import xgboost as xgb
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import datasets, linear_model
import scipy.sparse as sps
from scipy.sparse import coo_matrix, hstack, vstack, csr_matrix
from scipy import io
from datetime import datetime
import gc
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [100]:
def evalerror(preds, dtrain):

    labels = dtrain.get_label()
    assert len(preds) == len(labels)
    labels = labels.tolist()
    preds = preds.tolist()
    terms_to_sum = [(math.log(labels[i] + 1) - math.log(max(0,preds[i]) + 1))
                    ** 2.0 for i,pred in enumerate(labels)]
    return 'error', (sum(terms_to_sum) * (1.0/len(preds))) ** 0.5

In [101]:
products = pd.read_csv("../input/producto_tabla.csv")
products.head()

,Producto_ID,NombreProducto
0,0,NO IDENTIFICADO 0
1,9,Capuccino Moka 750g NES 9
2,41,Bimbollos Ext sAjonjoli 6p 480g BIM 41
3,53,Burritos Sincro 170g CU LON 53
4,72,Div Tira Mini Doradita 4p 45g TR 72


In [102]:
products['short_name'] = products.NombreProducto.str.extract('^(\D*)', expand=False)
# products['brand'] = products.NombreProducto.str.extract('^.+\s(\D+) \d+$', expand=False)
w = products.NombreProducto.str.extract('(\d+)(Kg|g)', expand=True)
products['weight'] = w[0].astype('float') * w[1].map({'Kg': 1000, 'g': 1})
products['pieces'] = products.NombreProducto.str.extract('(\d+)p ', expand=False).astype('float')

products['short_name_processed'] = (products['short_name'].
                                    map(lambda x: " ".
                                        join([i for i in x.lower().split() 
                                              if i not in nltk.corpus.stopwords.words("spanish")])))
stemmer = SnowballStemmer("spanish")
products['short_name_processed'] = (products['short_name_processed'].
                                    map(lambda x: " ".join([stemmer.stem(i) for i in x.lower().split()])))

word_list = products['short_name_processed']

word_counter = {}
for word in word_list:
    if word in word_counter:
        word_counter[word] += 1
    else:
        word_counter[word] = 1

popular_words = sorted(word_counter, key = word_counter.get, reverse = True)

short_name_processed_list = popular_words[:10]


# short_name_processed_list = products['short_name_processed'].unique()

products = pd.concat([products.drop(['short_name', 'short_name_processed', 'NombreProducto'], axis=1),
                      pd.get_dummies(short_name_processed_list)], axis=1)
#products = products.drop([''], axis=1)

products.fillna(value=0, inplace=True)

print('products shape:', products.shape)

('products shape:', (2592, 13))


In [103]:
train = pd.read_csv('../input/train.csv', usecols=['Semana', 'Cliente_ID', 'Producto_ID', 'Demanda_uni_equil'])
test = pd.read_csv('../input/test.csv', usecols=['id', 'Semana', 'Cliente_ID', 'Producto_ID'])
train.tail()

,Semana,Cliente_ID,Producto_ID,Demanda_uni_equil
74180459,9,4528866,32873,4
74180460,9,4528866,34226,4
74180461,9,4528866,45112,4
74180462,9,4547943,40217,0
74180463,9,4708097,43159,1


In [104]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74180464 entries, 0 to 74180463
Data columns (total 4 columns):
Semana               int64
Cliente_ID           int64
Producto_ID          int64
Demanda_uni_equil    int64
dtypes: int64(4)
memory usage: 2.2 GB


In [105]:
test.head()

,id,Semana,Cliente_ID,Producto_ID
0,0,11,4639078,35305
1,1,11,4705135,1238
2,2,10,4549769,32940
3,3,11,4717855,43066
4,4,11,966351,1277


In [106]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6999251 entries, 0 to 6999250
Data columns (total 4 columns):
id             int64
Semana         int64
Cliente_ID     int64
Producto_ID    int64
dtypes: int64(4)
memory usage: 213.6 MB


In [107]:
grouping = ['Semana', 'Cliente_ID', 'Producto_ID']
train_group = train.groupby(grouping,as_index=False)['Demanda_uni_equil'].sum()

In [108]:
train_group.tail()

,Semana,Cliente_ID,Producto_ID,Demanda_uni_equil
74013018,9,2015152015,1278,20
74013019,9,2015152015,2233,3
74013020,9,2015152015,2665,10
74013021,9,2015152015,4280,8
74013022,9,2015152015,31717,3


In [109]:
train_group.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74013023 entries, 0 to 74013022
Data columns (total 4 columns):
Semana               int64
Cliente_ID           int64
Producto_ID          int64
Demanda_uni_equil    int64
dtypes: int64(4)
memory usage: 2.8 GB


In [ ]:
train_group.head()

,Semana,Cliente_ID,Producto_ID,Demanda_uni_equil
0,3,26,1182,39
1,3,26,4767,42
2,3,26,31393,20
3,3,26,31690,42
4,3,26,32962,3


In [ ]:
train_cliprod = train_group
train_cliprod["cliprod"] = train_cliprod["Cliente_ID"].map(str) + ' ' + train_cliprod["Producto_ID"].map(str)

In [ ]:
test_cliprod = test
test_cliprod["cliprod"] = test_cliprod["Cliente_ID"].map(str) + ' ' + test_cliprod["Producto_ID"].map(str)

In [ ]:
train_cliprod.head()

In [ ]:
test_cliprod.head()

In [ ]:
grouping = ['Semana', 'cliprod']
train_cliprod = train_cliprod.groupby(grouping,as_index=False)['Demanda_uni_equil'].sum()


In [ ]:
train_cliprod.head()

In [ ]:
train_cliprod.tail()

In [ ]:
s3 = train_cliprod[train_cliprod.Semana == 3].drop('Semana', axis=1).set_index('cliprod')
s4 = train_cliprod[train_cliprod.Semana == 4].drop('Semana', axis=1).set_index('cliprod')
s5 = train_cliprod[train_cliprod.Semana == 5].drop('Semana', axis=1).set_index('cliprod')
s6 = train_cliprod[train_cliprod.Semana == 6].drop('Semana', axis=1).set_index('cliprod')
s7 = train_cliprod[train_cliprod.Semana == 7].drop('Semana', axis=1).set_index('cliprod')
s8 = train_cliprod[train_cliprod.Semana == 8].drop('Semana', axis=1).set_index('cliprod')
s9 = train_cliprod[train_cliprod.Semana == 9].drop('Semana', axis=1).set_index('cliprod')

t10 = test_cliprod[test_cliprod.Semana == 10].drop('Semana', axis=1).set_index('cliprod')
t11 = test_cliprod[test_cliprod.Semana == 11].drop('Semana', axis=1).set_index('cliprod')

In [ ]:
s3.head()

In [ ]:
t10.head()

In [ ]:
s4["d1"]=s3['Demanda_uni_equil']
s5["d1"]=s4['Demanda_uni_equil']
s6["d1"]=s5['Demanda_uni_equil']
s7["d1"]=s6['Demanda_uni_equil']
s8["d1"]=s7['Demanda_uni_equil']
s9["d1"]=s8['Demanda_uni_equil']

t10["d1"]=s9['Demanda_uni_equil']

s5["d2"]=s3['Demanda_uni_equil']
s6["d2"]=s4['Demanda_uni_equil']
s7["d2"]=s5['Demanda_uni_equil']
s8["d2"]=s6['Demanda_uni_equil']
s9["d2"]=s7['Demanda_uni_equil']

t10["d2"]=s8['Demanda_uni_equil']
t11["d2"]=s9['Demanda_uni_equil']

s6["d3"]=s3['Demanda_uni_equil']
s7["d3"]=s4['Demanda_uni_equil']
s8["d3"]=s5['Demanda_uni_equil']
s9["d3"]=s6['Demanda_uni_equil']

t10["d3"]=s7['Demanda_uni_equil']
t11["d3"]=s8['Demanda_uni_equil']

s7["d4"]=s3['Demanda_uni_equil']
s8["d4"]=s4['Demanda_uni_equil']
s9["d4"]=s5['Demanda_uni_equil']

t10["d4"]=s6['Demanda_uni_equil']
t11["d4"]=s7['Demanda_uni_equil']

s8["d5"]=s3['Demanda_uni_equil']
s9["d5"]=s4['Demanda_uni_equil']

t10["d5"]=s5['Demanda_uni_equil']
t11["d5"]=s6['Demanda_uni_equil']

s9["d6"]=s3['Demanda_uni_equil']

t10["d6"]=s4['Demanda_uni_equil']
t11["d6"]=s5['Demanda_uni_equil']

In [ ]:
s3.fillna(value=0, inplace=True)
s4.fillna(value=0, inplace=True)
s5.fillna(value=0, inplace=True)
s6.fillna(value=0, inplace=True)
s7.fillna(value=0, inplace=True)
s8.fillna(value=0, inplace=True)
s9.fillna(value=0, inplace=True)

t10.fillna(value=0, inplace=True)
t11.fillna(value=0, inplace=True)




In [ ]:
s3["d1"]=np.nan

t11["d1"]=np.nan


s3["d2"]=np.nan
s4["d2"]=np.nan



s3["d3"]=np.nan
s4["d3"]=np.nan
s5["d3"]=np.nan



s3["d4"]=np.nan
s4["d4"]=np.nan
s5["d4"]=np.nan
s6["d4"]=np.nan


s3["d5"]=np.nan
s4["d5"]=np.nan
s5["d5"]=np.nan
s6["d5"]=np.nan
s7["d5"]=np.nan



s3["d6"]=np.nan
s4["d6"]=np.nan
s5["d6"]=np.nan
s6["d6"]=np.nan
s7["d6"]=np.nan
s8["d6"]=np.nan



In [ ]:
s3.head()

In [ ]:
t10.head()

In [ ]:
s3 = s3.reset_index()
s3.head()

In [ ]:
s3['Cliente_ID'], s3['Producto_ID'] = zip(*s3['cliprod'].apply(lambda x: x.split(' ', 1)))

In [ ]:
s3.head()

In [ ]:
s3 = s3.drop(['cliprod'], axis=1)
s3.head()

In [ ]:
s4 = s4.reset_index()
s4['Cliente_ID'], s4['Producto_ID'] = zip(*s4['cliprod'].apply(lambda x: x.split(' ', 1)))
s4 = s4.drop(['cliprod'], axis=1)

s5 = s5.reset_index()
s5['Cliente_ID'], s5['Producto_ID'] = zip(*s5['cliprod'].apply(lambda x: x.split(' ', 1)))
s5 = s5.drop(['cliprod'], axis=1)

s6 = s6.reset_index()
s6['Cliente_ID'], s6['Producto_ID'] = zip(*s6['cliprod'].apply(lambda x: x.split(' ', 1)))
s6 = s6.drop(['cliprod'], axis=1)

s7 = s7.reset_index()
s7['Cliente_ID'], s7['Producto_ID'] = zip(*s7['cliprod'].apply(lambda x: x.split(' ', 1)))
s7 = s7.drop(['cliprod'], axis=1)

s8 = s8.reset_index()
s8['Cliente_ID'], s8['Producto_ID'] = zip(*s8['cliprod'].apply(lambda x: x.split(' ', 1)))
s8 = s8.drop(['cliprod'], axis=1)

s9 = s9.reset_index()
s9['Cliente_ID'], s9['Producto_ID'] = zip(*s9['cliprod'].apply(lambda x: x.split(' ', 1)))
s9 = s9.drop(['cliprod'], axis=1)


t10 = t10.reset_index()
t10['Cliente_ID'], t10['Producto_ID'] = zip(*t10['cliprod'].apply(lambda x: x.split(' ', 1)))
t10 = t10.drop(['cliprod'], axis=1)

t11 = t11.reset_index()
t11['Cliente_ID'], t11['Producto_ID'] = zip(*t11['cliprod'].apply(lambda x: x.split(' ', 1)))
t11 = t11.drop(['cliprod'], axis=1)

In [ ]:
t10.head()

In [ ]:
train_joined = pd.concat([s3, s4, s5, s6, s7, s8, s9], ignore_index = True)
test_joined = pd.concat([t10, t11], ignore_index = True)

In [ ]:
train_joined.info()

In [ ]:
train_joined.head()

In [ ]:
train_joined[['Cliente_ID', 'Producto_ID']]=train_joined[['Cliente_ID', 'Producto_ID']].astype('int32')
test_joined[['Cliente_ID', 'Producto_ID']]=test_joined[['Cliente_ID', 'Producto_ID']].astype('int32')

In [ ]:
train_joined.info()

In [ ]:
test_joined.info()

In [ ]:
train_joined.to_csv('../input/train_alld.csv')
test_joined.to_csv('../input/test_alld.csv')
train_joined.fillna(value=-999, inplace=True)
test_joined.fillna(value=-999, inplace=True)

In [ ]:
# Join data and products
train_joined = train_joined.join(products, on='Producto_ID', lsuffix='_t')
train_joined.fillna(value=0, inplace=True)

print('train data joined')

test_joined = test_joined.join(products, on='Producto_ID', lsuffix='_t')
test_joined.fillna(value=0, inplace=True)

print('test data joined')
print('train shape', train_joined.shape)
print('test shape', test_joined.shape)

In [ ]:
ids = test_joined['id']
test_joined = test_joined.drop(['id'],axis = 1)

y = train_joined['Demanda_uni_equil']
X = train_joined[test_joined.columns.values]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1729)

print ('Division_Set_Shapes:', X.shape, y.shape)
print ('Validation_Set_Shapes:', X_train.shape, X_test.shape)

In [ ]:
X_train.info()

In [ ]:
params = {}
params['objective'] = "reg:linear"
params['eta'] = 0.025
params['max_depth'] = 5
params['subsample'] = 0.8
params['colsample_bytree'] = 0.6
params['silent'] = True

In [ ]:
test_preds = np.zeros(test_joined.shape[0])
xg_train = xgb.DMatrix(X_train, label=y_train, missing = -999)
xg_test = xgb.DMatrix(X_test)

watchlist = [(xg_train, 'train_joined')]
num_rounds = 100

xgclassifier = xgb.train(params, xg_train, num_rounds, watchlist, feval = evalerror,
                         early_stopping_rounds= 20, verbose_eval = 10)
preds = xgclassifier.predict(xg_test, ntree_limit=xgclassifier.best_iteration)

print ('RMSLE Score:', rmsle(y_test, preds))

In [ ]:
fxg_test = xgb.DMatrix(test_joined)
fold_preds = np.around(xgclassifier.predict(fxg_test, ntree_limit=xgclassifier.best_iteration), decimals = 1)
test_preds += fold_preds

In [ ]:
submission = pd.DataFrame({'id':ids, 'Demanda_uni_equil': test_preds})

In [ ]:
submission.Demanda_uni_equil = submission.Demanda_uni_equil.round()

In [ ]:
submission[["id","Demanda_uni_equil"]].to_csv('../submissions/' +
                                              datetime.now().strftime('%Y-%m-%d-%H-%M-%S') +'.csv', index=False)

print ('done')